In [4]:
from dataclasses import dataclass
import random
import numpy as np

In [5]:
from typing import List

@dataclass
class XYZ:
    x: int
    y: int
    z: int

    @property
    def coords(self):
        return [self.x, self.y, self.z]

    def __add__(self, o):
        return XYZ(x = self.x + o.x, y = self.y + o.y, z = self.z + o.z)

    def __mul__(self, scalar):
        return XYZ(x = self.x * scalar, y = self.y * scalar, z = self.z * scalar)
        
    def __sub__(self, o):
        return self + (o * -1)

DOWN    = XYZ( 0, -1,  0)
UP      = XYZ( 0,  1,  0)
LEFT    = XYZ(-1,  0,  0)
RIGHT   = XYZ( 1,  0,  0)
BACK    = XYZ( 0,  0, -1)
FORWARD = XYZ( 0,  0,  1)

In [6]:
def parse_XYZ(line):
    parts = [int(x.strip()) for x in line.strip().split(', ')]
    return XYZ(*parts)

In [7]:
@dataclass
class Ray:
    start: XYZ
    velocity: XYZ

def parse_Ray(line):
    parts = line.strip().split(' @ ')
    return Ray(start=parse_XYZ(parts[0]), velocity=parse_XYZ(parts[1]))

In [8]:
with open('input', 'r') as f:
    input_file = f.readlines()

In [9]:
rays = [parse_Ray(line) for line in input_file]

In [120]:
flat_order = [BACK, RIGHT, FORWARD, LEFT]
side_order = [LEFT, BACK, RIGHT, FORWARD]

def flat_yield(start, size):
    curr = start
    yield curr
    step_size = 1
    ord_idx = 0
    while step_size <= size:
        for _a in range(0, 2):
            for _b in range(0, step_size):
                curr += flat_order[ord_idx]
                yield curr
            ord_idx += 1
            ord_idx %= 4
        step_size += 1
    for _a in range(0, size):
        curr += flat_order[ord_idx]
        yield curr


def cube_points_iterator(max_size=None, start_size=2):
    size = start_size
    curr = XYZ(0, 1, 0) * (size // 2)
    while True:
        if max_size is not None and max_size < size:
            break
        top_point = curr + UP

        # Flat top
        yield from flat_yield(top_point, size)

        # Sides
        front_right_point = top_point + FORWARD * (size // 2) + RIGHT * (size // 2)
        curr = front_right_point
        for side in range(1, size):
            curr += DOWN
            yield curr
            for ord_idx in range(0, 4):
                for _i in range(1, size + 1):
                    curr += side_order[ord_idx]
                    yield curr

        # Flat bottom
        bottom_point = top_point + (DOWN * (size))
        yield from flat_yield(bottom_point, size)

        # End
        curr = top_point
        print('Past all points in cube of size: ', size)
        size += 2

In [125]:
def simpler_iterator(min_x, max_x, min_y, max_y, min_z, max_z):
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            for z in range(min_z, max_z + 1):
                yield XYZ(x, y, z)

In [124]:
def test_vel_yield(size, start_size=2):
    list_vel = list(cube_points_iterator(size))
    maximum_ss = (start_size // 2)
    minimum = -(size // 2)
    maximum = (size // 2)

    for x in range(minimum, maximum +1):
        for y in range(minimum, maximum +1):
            for z in range(minimum, maximum +1):
                if abs(x) <maximum_ss or abs(y) < maximum_ss or abs(z) < maximum_ss:
                    continue
                punkt = XYZ(x, y, z)
                assert punkt in list_vel, f'Missing {punkt} in {list_vel}'

test_vel_yield(2)
test_vel_yield(4)
test_vel_yield(16, 8)

Past all points in cube of size:  2
Past all points in cube of size:  2
Past all points in cube of size:  4
Past all points in cube of size:  2
Past all points in cube of size:  4
Past all points in cube of size:  6
Past all points in cube of size:  8
Past all points in cube of size:  10
Past all points in cube of size:  12
Past all points in cube of size:  14
Past all points in cube of size:  16


In [102]:
# 0: s.x, 1: s.y, 2: s.z, 3: t_0, .... 

def equations_for(co_len, velocity, coord, rays: List[Ray], how_many):
    coefficients = []
    results = []
    for i in range(0, min(how_many, len(rays))):
        ray = rays[i]
        cos = [0] * co_len
        cos[coord] = 1
        t_idx = 3 + i
        cos[t_idx] = velocity.coords[coord] - ray.velocity.coords[coord]
        coefficients.append(cos)
        results.append(ray.start.coords[coord])
    return (coefficients, results)

def compute_for(velocity, rays):
    coefficients = []
    results = []

    co_len = len(rays) + 3
    left = co_len*3
    e, r = equations_for(co_len, velocity, 0, rays, left)
    coefficients.extend(e)
    results.extend(r)

    left -= len(r)
    e, r = equations_for(co_len, velocity, 1, rays, left)
    coefficients.extend(e)
    results.extend(r)

    left -= len(r)
    e, r = equations_for(co_len, velocity, 2, rays, left)
    coefficients.extend(e)
    results.extend(r)

    c = np.array(coefficients)
    r = np.array(results)

    res, _, _, _ = np.linalg.lstsq(c, r)
    return res

In [112]:
def verify(res, velocity: XYZ, rays: List[Ray]):
    for coord_idx in range(0, 3):
        s = res[coord_idx]
        s_v = velocity.coords[coord_idx]
        for ray_idx, ray in enumerate(rays):
            t_idx = 3 + ray_idx
            t = res[t_idx]
            #print("l, r:", s + t* (s_v - ray.velocity.coords[coord_idx]), ray.start.coords[coord_idx])
            if not np.isclose(s + t* (s_v - ray.velocity.coords[coord_idx]), ray.start.coords[coord_idx], rtol=0.001, atol=0.001):
                return False
    return True

In [121]:
def find_solution(rays, num):
    for vel in cube_points_iterator():
        sample_rays = random.sample(rays, num)
        res = compute_for(vel, sample_rays)
        if verify(res, vel, sample_rays):
            start = XYZ(res[0], res[1], res[2])
            print("Start: ", start)
            print("Velocity: ", vel)
            return

In [114]:
test_input = """1, 1, 0 @ -1, 0, 0
2, 2, 0 @ -1, 0, 0
3, 3, 0 @ -1, 0, 0
4, 4, 0 @ -1, 0, 0
-5, -5, 0 @ 1, 0, 0"""
# Res: 0, 0, 0 @ 0, 1, 0

test_rays = [parse_Ray(line) for line in test_input.split('\n')]

In [115]:
res = compute_for(XYZ(0, 1, 0), test_rays)

/tmp/ipykernel_601/3557056552.py:39: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  res, _, _, _ = np.linalg.lstsq(c, r)


In [118]:
find_solution(test_rays, len(test_rays))

Start:  XYZ(x=-4.999999999999995, y=-4.9999999999999964, z=-8.881784197001252e-16)
Velocity:  XYZ(x=0, y=1, z=0)


/tmp/ipykernel_601/3557056552.py:39: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  res, _, _, _ = np.linalg.lstsq(c, r)


In [122]:
find_solution(rays, len(rays))

/tmp/ipykernel_601/3557056552.py:39: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  res, _, _, _ = np.linalg.lstsq(c, r)


Past all points in cube of size:  2
Past all points in cube of size:  4
Past all points in cube of size:  6
Past all points in cube of size:  8
Past all points in cube of size:  10
Past all points in cube of size:  12
Past all points in cube of size:  14
Past all points in cube of size:  16
Past all points in cube of size:  18
Past all points in cube of size:  20
Past all points in cube of size:  22
Past all points in cube of size:  24
Past all points in cube of size:  26
Past all points in cube of size:  28
Past all points in cube of size:  30
Past all points in cube of size:  32
Past all points in cube of size:  34
Past all points in cube of size:  36


KeyboardInterrupt: 

In [126]:
guess_vel = XYZ(245,75,221)
guess_res = compute_for(guess_vel, rays)
verify(guess_res, guess_vel, rays)

/tmp/ipykernel_601/3557056552.py:39: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  res, _, _, _ = np.linalg.lstsq(c, r)


True

In [127]:
guess_res

array([1.59153037e+14, 2.28139154e+14, 1.70451316e+14, 7.34248440e+11,
       8.54610412e+11, 3.00825392e+11, 8.91181836e+11, 8.31808607e+11,
       8.71156047e+11, 5.01798422e+11, 9.84569048e+11, 7.95502862e+11,
       9.51688371e+11, 5.32442826e+11, 8.64175465e+11, 4.41167286e+11,
       4.21699386e+11, 3.36662694e+11, 2.16550982e+11, 1.03207228e+12,
       3.13040513e+11, 8.52308161e+11, 6.50693441e+11, 1.80294721e+11,
       5.36791923e+11, 4.25357333e+11, 6.20395946e+11, 8.16569411e+10,
       4.18757631e+11, 1.01543035e+11, 6.68784853e+11, 7.81008798e+11,
       1.53541661e+11, 5.66348222e+10, 1.01093378e+12, 3.65262950e+11,
       8.73685197e+11, 7.25482658e+11, 9.91315785e+11, 5.88703883e+10,
       3.96464019e+11, 4.95492050e+11, 4.53581633e+11, 5.45523664e+11,
       5.54451045e+11, 4.67166712e+11, 1.44333741e+11, 6.62435239e+10,
       8.24028168e+11, 5.10309634e+11, 9.00655440e+11, 4.93678155e+11,
       3.10883141e+11, 8.69814230e+11, 9.87253317e+11, 6.10243488e+11,
      

In [129]:
guess_res[0] + guess_res[1] + guess_res[2]

557743507346378.94

In [130]:
round(guess_res[0] + guess_res[1] + guess_res[2])

557743507346379

In [134]:
from sympy import Symbol
import sympy

# S.x + t * S.vx = R.x + t * R.vx
# S.x + t * (S.vx - R.vx) - R.x = 0

Sx = Symbol('Sx')
Sy = Symbol('Sy')
Sz = Symbol('Sz')

Svx = Symbol('Svx')
Svy = Symbol('Svy')
Svz = Symbol('Svz')

equations = []

for i, ray in enumerate(random.sample(rays, 3)):
    t = Symbol(f't_{i}')
    equations.append(Sx + t * (Svx - ray.velocity.x) - ray.start.x)
    equations.append(Sy + t * (Svy - ray.velocity.y) - ray.start.y)
    equations.append(Sz + t * (Svz - ray.velocity.z) - ray.start.z)

solution = sympy.solve(equations, dict=True)

In [145]:
solution[0][Sx] + solution[0][Sy] + solution[0][Sz]

557743507346379

In [146]:
solution

[{Svx: 245,
  Svy: 75,
  Svz: 221,
  Sx: 159153037374407,
  Sy: 228139153674672,
  Sz: 170451316297300,
  t_0: 517971898398,
  t_1: 726407210839,
  t_2: 900655440059}]